In [ ]:
from pathlib import Path

from matflow import load_workflow

import numpy as np
import matplotlib.pyplot as plt

%matplotlib widget

## Load workflow

In [ ]:
workflow_path = Path('/home/mike/Desktop/csf-home/scratch/RVE_extrusion_2021-05-03-235442/')
workflow = load_workflow(workflow_path)

sim_task = workflow.tasks.simulate_volume_element_loading
vol_elem_resp = sim_task.elements[0].outputs.volume_element_response

### Plot stress strain curve

In [ ]:
mean_strain = vol_elem_resp['incremental_data']['vol_avg_strain']['data']
mean_stress = vol_elem_resp['incremental_data']['vol_avg_stress']['data']

In [ ]:
plt.figure()
plt.plot(mean_strain[:, 0, 0], mean_stress[:, 0, 0]/1e6)

plt.xlabel('Strain')
plt.ylabel('Stress (MPa)')
plt.xlim([0, None])
plt.ylim([0, None])

### Plot stress map

In [ ]:
stress = vol_elem_resp['field_data']['sigma']
print(stress['meta']['increments'])
stress = stress['data']
print(stress.shape)

In [ ]:
inc = 2
z_layer = 2
tensor_comp = (0, 0)
buff_size_xy = (4, 4)

map_slice = (
    inc, 
    slice(buff_size_xy[0], -buff_size_xy[0]), 
    slice(buff_size_xy[1], -buff_size_xy[1]), 
    z_layer
) + tensor_comp

plt.figure()
plt.imshow(stress[map_slice].T*1e-6, vmin=250, vmax=400)
plt.colorbar(label='Stress (MPa)')
plt.xticks([])
plt.yticks([])

plt.tight_layout()

### Plot strain map

In [ ]:
strain = vol_elem_resp['field_data']['epsilon_U^2(F)']
print(strain['meta']['increments'])
strain = strain['data']
print(strain.shape)

In [ ]:
inc = 2
z_layer = 2
tensor_comp = (0, 0)
buff_size_xy = (4, 4)

map_slice = (
    inc, 
    slice(buff_size_xy[0], -buff_size_xy[0]), 
    slice(buff_size_xy[1], -buff_size_xy[1]), 
    z_layer
) + tensor_comp

plt.figure()
plt.imshow(strain[map_slice].T)
plt.colorbar(label='Strain')
plt.xticks([])
plt.yticks([])

plt.tight_layout()

### Plot grain average stress

In [ ]:
stress_grain = vol_elem_resp['grain_data']['sigma']
print(stress_grain['meta']['increments'])
stress_grain = stress_grain['data']
print(stress_grain.shape)

In [ ]:
tensor_comp = (0, 0)

plt.figure()
for inc in range(3):
    # Remove last 2 grain of air and iso Ti
    data_slice = (inc, slice(None, -2)) + tensor_comp

    plt.hist(stress_grain[data_slice]*1e-6, histtype='step', density=True, label=f'inc {inc}')
    
plt.legend()
plt.xlabel('Stress (MPa)')
plt.ylabel('Normalised frequency')

### Plot grain average orientations

In [ ]:
from defdap.quat import Quat

In [ ]:
ori_grain = vol_elem_resp['grain_data']['O']
print(ori_grain['meta']['increments'])
ori_grain = ori_grain['data']['quaternions']
print(ori_grain.shape)

In [ ]:
# Swap to P=+1
ori_grain[:, :, 1:] *= -1
# Remove last 2 grain of air and iso Ti
ori_grain = ori_grain[:, :-2, :]

In [ ]:
quat_grain = np.empty(ori_grain.shape[:-1], dtype=Quat)

for idx in np.ndindex(quat_grain.shape):
    quat_grain[idx] = Quat(ori_grain[idx])

In [ ]:
inc = 0
direction = np.array([1, 0, 0])

Quat.plotIPF(quat_grain[inc], direction, 'cubic')